In [20]:
# %load ../20-00-data-wrangle-MODIS-NDVI/10-NDVI-daily-averages/workerbee.py
import os
import sys
import pickle
import geopandas as gpd
import rasterio as rio
from rasterio.windows import from_bounds
from playLib.log_logger import log_make_logger
from playLib.bo_bucket import s3cp



def clip_by_geojson(infile,outfile,geojson_file):

    filepath=infile
    geoms = gpd.read_file(geojson_file)
    print(geoms)
    left = geoms.bounds.minx[0]
    right = geoms.bounds.maxx[0]
    top = geoms.bounds.maxy[0]
    bottom = geoms.bounds.miny[0]

    print(left,right,top,bottom)

    with rio.open(filepath) as src:
        my_window = from_bounds(left, bottom, right, top, src.transform)
        print(my_window)

        rst = src.read(1, window=my_window)

        py, px = src.index(left, top)
        print('Pixel Y, X coords: {}, {}'.format(py, px))

        py, px = src.index(-90, 49.98)
        print('Pixel Y, X coords: {}, {}'.format(py, px))
        rst[(rst < 0)] = 0

        # You can then write out a new file
        meta = src.meta
        print(meta)
        #meta['width'], meta['height'] = rst.shape[1],rst.shape[0]
        meta['width'], meta['height'] = rst.shape[0],rst.shape[1]

        meta['transform'] = rio.windows.transform(my_window, src.transform)
        print(meta)

        with rio.open(outfile, 'w', **meta) as dst:
        #with rio.open(outfile, 'w', **meta, compress="DEFLATE",
                   #tiled=True, blockxsize=256, blockysize=256) as dst:
            dst.write(rst, 1)

        return rst


log=log_make_logger('WORKERBEE')
log.info('starting whatever YAY!')

geoj='great_lakes_edit.geojson'

infile = 's3://' + 'dev-et-data/compressed/NDVI_filled/2001/' + '2001180.250_m_NDVI.tif'

out= 's3://' + 'dev-et-data/tony/' + 'gl_stripped.tif'

# out= 's3://' + 'dev-et-data/tony/' + 'gl_cog.tif'


temp='a.tif'

clip_by_geojson(infile,temp,geoj)

s3cp(temp,out)
os.remove(temp)
print("{} File Removed!".format(temp))


2020-05-16 18:48:53,359 - WORKERBEE - INFO - Logging Begins
2020-05-16 18:48:53,359 - WORKERBEE - INFO - Logging Begins
2020-05-16 18:48:53,362 - WORKERBEE - INFO - starting whatever YAY!
2020-05-16 18:48:53,362 - WORKERBEE - INFO - starting whatever YAY!


                                            geometry
0  POLYGON ((-90.00000 49.90000, -80.00000 49.900...
-90.0 -80.0 49.9 40.0
Window(col_off=31509.966777102105, row_off=48.053714444195066, width=4805.371636630291, height=4757.317920263988)
Pixel Y, X coords: 48, 31509
Pixel Y, X coords: 9, 31509
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 49667, 'height': 14416, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0020810045, 0.0, -155.57238265799998,
       0.0, -0.0020810045, 49.999999996000085)}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 4757, 'height': 4805, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0020810045, 0.0, -90.0,
       0.0, -0.0020810045, 49.9)}
hello from s3get copying file a.tif
aws s3 cp a.tif s3://dev-et-data/tony/gl_stripped.tif
a.tif File Removed!


In [27]:
! gdalinfo /vsis3/dev-et-data/tony/gl_cog.tif | egrep 'COMPRESSION|Block'

  COMPRESSION=DEFLATE
Band 1 Block=256x256 Type=Float32, ColorInterp=Gray


In [28]:
! ! gdalinfo /vsis3/dev-et-data/tony/gl_stripped.tif | egrep 'COMPRESSION|Block'

Band 1 Block=4757x1 Type=Float32, ColorInterp=Gray


In [14]:
#!echo .tif >>.gitignore
! cat .gitignore

my_cred.json
.tif
.tif


In [29]:
! aws s3 ls dev-et-data/compressed/NDVI_filled/2001/ | grep 2001180

2020-04-30 01:09:07 1171781350 2001180.250_m_NDVI.tif
2020-04-30 01:09:19       2816 2001180.250_m_NDVI.tif.aux.xml


In [32]:
# Show the geometry of scene vs AOI against a basemap
import folium
geoms = gpd.read_file('great_lakes_edit.geojson')
m = folium.Map([50, -90], zoom_start=7, width=900, height=350, tiles='OpenStreetMap')
folium.GeoJson(geoms).add_to(m)
#m.add_child(Marker(location=[45.3676, -97.4048], popup='Waubay, South Dakota AOI', icon = folium.Icon(color = 'red')))
#folium.LatLngPopup().add_to(m)
m